In [1]:
import pandas as pd
import requests
import re
import html
from bs4 import BeautifulSoup
import time

# Configuration
BASE_URL = "https://www.bcb.gov.br/api/servico/sitebcb/copom/comunicados_detalhes?nro_reuniao={}"

In [2]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def fetch_copom_data(meeting_number):
    """Fetches raw data for a specific meeting number."""
    url = BASE_URL.format(meeting_number)
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "application/json"
    }
    try:
        # BCB API often has certificate issues, so we skip verification
        response = requests.get(url, headers=headers, timeout=30, verify=False)
        response.raise_for_status()
        data = response.json()
        
        if not data:
            return None

        # Check for "conteudo" key (new API structure)
        if isinstance(data, dict) and "conteudo" in data:
            items = data["conteudo"]
            if items and len(items) > 0:
                return items[0]
        
        # Fallback for list structure
        if isinstance(data, list) and len(data) > 0:
            return data[0]
            
        return None
    except Exception as e:
        print(f"Error fetching meeting {meeting_number}: {e}")
        return None

def clean_html(raw_html):
    """Parses HTML, extracts text and tables."""
    if not raw_html:
        return "", []

    decoded = html.unescape(raw_html)
    # Remove zero-width spaces and other artifacts
    decoded = re.sub(r'[\u200b\ufeff\u00a0]', ' ', decoded)
    soup = BeautifulSoup(decoded, 'html.parser')

    tables = []
    # Extract tables
    for i, table in enumerate(soup.find_all('table')):
        rows = []
        for tr in table.find_all('tr'):
            cells = [td.get_text(strip=True) for td in tr.find_all(['td', 'th'])]
            if any(cells): # Only add non-empty rows
                rows.append(cells)
        if rows:
            df_table = pd.DataFrame(rows)
            tables.append(df_table)
        
        # Look back for titles/labels (e.g. "Tabela 1", "Projeções...")
        # We check up to 3 preceding elements
        for _ in range(3):
            prev = table.find_previous_sibling()
            
            # Skip whitespace/strings
            while prev and isinstance(prev, str) and not prev.strip():
                prev = prev.find_previous_sibling()
            
            if prev and prev.name in ['p', 'div', 'span', 'strong', 'b', 'h4', 'h5', 'h6']:
                text_content = prev.get_text(strip=True)
                # Heuristic: Titles are usually short (< 150 chars)
                # And shouldn't end with a period usually, but some do.
                # We'll stick to length as primary heuristic.
                if len(text_content) < 150:
                    prev.decompose()
                else:
                    break # Stop if we hit a long paragraph
            else:
                break # Stop if we hit something else or nothing

        table.decompose() # Remove table from soup to avoid duplicating text

    # formatting
    for br in soup.find_all('br'):
        br.replace_with('\n')
    for p in soup.find_all('p'):
        p.insert_after('\n\n')

    text = soup.get_text()
    # Clean up whitespace
    text = re.sub(r'\n\s*\n', '\n\n', text)
    text = re.sub(r'[ \t]+', ' ', text)
    text = text.strip()

    return text, tables

In [3]:
def extract_metadata(text, title, date_ref):
    """Extracts structured data from the text using Regex."""
    metadata = {
        "selic_rate": None,
        "decision": None,
        "num_directors": None,
        "directors": [],
        "board_president": None,
        "score": None,
    }

    # 1. Selic Rate
    # Patterns: "Selic para X,XX%", "Selic em X,XX%"
    selic_match = re.search(r'Selic\s+(?:para|em|de)\s+(\d+(?:[,\.]\d+)?)\s*%', text, re.IGNORECASE)
    if selic_match:
        metadata["selic_rate"] = selic_match.group(1).replace(',', '.')

    # 2. Decision
    text_lower = text.lower()
    if "manteve" in text_lower or "manter" in text_lower:
        metadata["decision"] = "maintain"
    elif "elevou" in text_lower or "elevar" in text_lower or "aumentou" in text_lower:
        metadata["decision"] = "hike"
    elif "reduziu" in text_lower or "reduzir" in text_lower:
        metadata["decision"] = "cut"

    # 3. Score / Unanimity
    if "unanimidade" in text_lower:
        metadata["score"] = "unanimous"
    else:
        # Try to find "X votos a Y"
        vote_match = re.search(r'(\d+)\s+votos?\s+a\s+(\d+)', text_lower)
        if vote_match:
            metadata["score"] = f"{vote_match.group(1)}x{vote_match.group(2)}"

    # 4. Directors & President
    # Simple extraction of President
    pres_match = re.search(r'Presidente:\s*([A-Z][a-záéíóúàâêôãõç\s\.]+)(?:\.|,|$)', text)
    if pres_match:
        metadata["board_president"] = pres_match.group(1).strip()

    # Try to extract list of directors (heuristic)
    # Look for the section starting with "Votaram por..." or "Membros do Copom presentes:"
    members_match = re.search(r'(?:Votaram por|Membros do Copom presentes)[:\s]+([\s\S]+?)(?:\n\n|$)', text)
    if members_match:
        members_text = members_match.group(1)
        # Split by commas or 'e'
        names = re.split(r',|\se\s', members_text)
        clean_names = [n.strip().strip('.') for n in names if len(n.strip()) > 3 and n.strip()[0].isupper()]
        metadata["directors"] = clean_names
        metadata["num_directors"] = len(clean_names)

    return metadata

In [4]:
def build_dataset(start_meeting, end_meeting):
    meetings_data = []
    paragraphs_data = []
    tables_data = []

    for meeting_num in range(start_meeting, end_meeting + 1):
        print(f"Processing meeting {meeting_num}...")
        raw_data = fetch_copom_data(meeting_num)
        
        if not raw_data:
            print(f"  No data found for {meeting_num}")
            continue

        # Basic info
        date_ref = raw_data.get('dataReferencia')
        title = raw_data.get('titulo')
        raw_html = raw_data.get('textoComunicado') or raw_data.get('conteudoHtml')

        # Clean and Parse
        full_text, tables = clean_html(raw_html)
        
        # Extract Metadata
        meta = extract_metadata(full_text, title, date_ref)

        # Store Meeting Level Data
        meeting_entry = {
            "meeting_number": meeting_num,
            "date": date_ref,
            "title": title,
            "full_text": full_text,
            **meta
        }
        meetings_data.append(meeting_entry)

        # Store Paragraphs
        # Split by double newlines
        paras = [p.strip() for p in full_text.split('\n\n') if p.strip()]
        for i, p in enumerate(paras):
            paragraphs_data.append({
                "meeting_number": meeting_num,
                "paragraph_id": i,
                "text": p,
                "char_count": len(p)
            })

        # Store Tables
        for i, tbl in enumerate(tables):
            tables_data.append({
                "meeting_number": meeting_num,
                "table_id": i,
                "dataframe": tbl
            })
        
        time.sleep(0.5) # Be nice to the API

    return pd.DataFrame(meetings_data), pd.DataFrame(paragraphs_data), tables_data

# Run for a sample range (e.g., last 10 meetings)
# Assuming current is around 274
df_meetings, df_paragraphs, list_tables = build_dataset(255, 274)

Processing meeting 255...
Processing meeting 256...
Processing meeting 256...
Processing meeting 257...
Processing meeting 257...
Processing meeting 258...
Processing meeting 258...
Processing meeting 259...
Processing meeting 259...
Processing meeting 260...
Processing meeting 260...
Processing meeting 261...
Processing meeting 261...
Processing meeting 262...
Processing meeting 262...
Processing meeting 263...
Processing meeting 263...
Processing meeting 264...
Processing meeting 264...
Processing meeting 265...
Processing meeting 265...
Processing meeting 266...
Processing meeting 266...
Processing meeting 267...
Processing meeting 267...
Processing meeting 268...
Processing meeting 268...
Processing meeting 269...
Processing meeting 269...
Processing meeting 270...
Processing meeting 270...
Processing meeting 271...
Processing meeting 271...
Processing meeting 272...
Processing meeting 272...
Processing meeting 273...
Processing meeting 273...
Processing meeting 274...
Processing m

In [5]:
# Display Results
print("Meetings DataFrame:")
display(df_meetings.tail())

print("\nParagraphs DataFrame:")
display(df_paragraphs.head())

print(f"\nTotal Tables Extracted: {len(list_tables)}")
if list_tables:
    print("Example Table:")
    display(list_tables[-1]['dataframe'])

Meetings DataFrame:


,meeting_number,date,title,full_text,selic_rate,decision,num_directors,directors,board_president,score
15,270,2025-05-07,"Copom eleva a taxa Selic para 14,75% a.a.",O ambiente externo mostra-se adverso e particu...,None,maintain,8,"[Ailton de Aquino Santos, Diogo Abry Guillen, ...",None,None
16,271,2025-06-18,"Copom eleva a taxa Selic para 15,00% a.a.",O ambiente externo mantém-se adverso e particu...,None,hike,8,"[Ailton de Aquino Santos, Diogo Abry Guillen, ...",None,None
17,272,2025-07-30,"Copom mantém a taxa Selic em 15,00% a.a.",O ambiente externo está mais adverso e incerto...,None,maintain,8,"[Ailton de Aquino Santos, Diogo Abry Guillen, ...",None,None
18,273,2025-09-17,"Copom mantém a taxa Selic em 15,00% a.a.",O ambiente externo se mantém incerto em função...,None,maintain,8,"[Ailton de Aquino Santos, Diogo Abry Guillen, ...",None,None
19,274,2025-11-05,"Copom mantém a taxa Selic em 15,00% a.a.",O ambiente externo ainda se mantém incerto em ...,None,maintain,8,"[Ailton de Aquino Santos, Diogo Abry Guillen, ...",None,None



Paragraphs DataFrame:


,meeting_number,paragraph_id,text,char_count
0,255,0,"O ambiente externo se mantém adverso, ainda qu...",478
1,255,1,"Em relação ao cenário doméstico, o conjunto do...",745
2,255,2,As projeções de inflação do Copom em seu cenár...,194
3,255,3,"O Comitê ressalta que, em seus cenários para a...",1162
4,255,4,"Considerando os cenários avaliados, o balanço ...",522



Total Tables Extracted: 10
Example Table:


,0,1,2,3
0,Índice de preços,2025,2026,2º tri 2027
1,IPCA,"4,6","3,6","3,3"
2,IPCA livres,"4,5","3,6","3,2"
3,IPCA administrados,"5,0","3,4","3,5"


In [6]:
# Final check of paragraph 8 for meeting 274
p8 = df_paragraphs[(df_paragraphs['meeting_number'] == 274) & (df_paragraphs['paragraph_id'] == 8)]
if not p8.empty:
    print(f"Current Paragraph 8 (Meeting 274):\n{p8.iloc[0]['text']}")
else:
    print("Paragraph 8 not found for meeting 274")

Current Paragraph 8 (Meeting 274):
No cenário de referência, a trajetória para a taxa de juros é extraída da pesquisa Focus e a taxa de câmbio parte de R$5,40/US$, evoluindo segundo a paridade do poder de compra (PPC). O preço do petróleo segue aproximadamente a curva futura pelos próximos seis meses e passa a aumentar 2% ao ano posteriormente. Além disso, adota-se a hipótese de bandeira tarifária “amarela" em dezembro de 2025 e de 2026. O valor para o câmbio foi obtido pelo procedimento usual.


In [7]:
# Inspect paragraph 8 of meeting 274
p8_274 = df_paragraphs[(df_paragraphs['meeting_number'] == 274) & (df_paragraphs['paragraph_id'] == 8)]
print("Paragraph 8 content:")
print(p8_274['text'].values[0] if not p8_274.empty else "Not found")

# Inspect HTML structure for meeting 274
raw_274 = fetch_copom_data(274)
if raw_274:
    html_274 = raw_274.get('textoComunicado') or raw_274.get('conteudoHtml')
    soup = BeautifulSoup(html_274, 'html.parser')
    tables = soup.find_all('table')
    print(f"\nFound {len(tables)} tables in meeting 274")
    
    for i, tbl in enumerate(tables):
        print(f"\n--- Table {i} ---")
        # Look at previous siblings
        curr = tbl.previous_sibling
        count = 0
        while curr and count < 5: # Look back a few steps
            if isinstance(curr, str):
                if curr.strip():
                    print(f"Prev sibling {count} (text): {curr.strip()[:100]}")
            else:
                print(f"Prev sibling {count} ({curr.name}): {curr.get_text(strip=True)[:100]}")
            curr = curr.previous_sibling
            count += 1

Paragraph 8 content:
No cenário de referência, a trajetória para a taxa de juros é extraída da pesquisa Focus e a taxa de câmbio parte de R$5,40/US$, evoluindo segundo a paridade do poder de compra (PPC). O preço do petróleo segue aproximadamente a curva futura pelos próximos seis meses e passa a aumentar 2% ao ano posteriormente. Além disso, adota-se a hipótese de bandeira tarifária “amarela" em dezembro de 2025 e de 2026. O valor para o câmbio foi obtido pelo procedimento usual.

Found 1 tables in meeting 274

--- Table 0 ---
Prev sibling 0 (p): Projeções de inflação no cenário de referência
Prev sibling 1 (p): Tabela 1
Prev sibling 2 (p): Votaram por essa decisão os seguintes membros do Comitê: Gabriel Muricca Galípolo (presidente), Ailt
Prev sibling 3 (p): O cenário atual, marcado por elevada incerteza, exige cautela na condução da política monetária. O C
Prev sibling 4 (p): O Copom decidiu manter a taxa básica de juros em 15,00% a.a., e entende que essa decisão é compatíve


In [8]:
df_paragraphs[df_paragraphs['meeting_number'] == 274]

,meeting_number,paragraph_id,text,char_count
192,274,0,O ambiente externo ainda se mantém incerto em ...,270
193,274,1,"Em relação ao cenário doméstico, o conjunto do...",367
194,274,2,As expectativas de inflação para 2025 e 2026 a...,324
195,274,3,"Os riscos para a inflação, tanto de alta quant...",960
196,274,4,O Comitê segue acompanhando os anúncios refere...,643
197,274,5,O Copom decidiu manter a taxa básica de juros ...,394
198,274,6,"O cenário atual, marcado por elevada incerteza...",452
199,274,7,Votaram por essa decisão os seguintes membros ...,295
200,274,8,"No cenário de referência, a trajetória para a ...",464


In [9]:
# Join all tables horizontally
dfs_to_join = []

for item in list_tables:
    meeting_num = item['meeting_number']
    table_id = item['table_id']
    df = item['dataframe'].copy()
    
    # Basic cleanup: Promote first row to header if it looks like a header
    # (The extraction logic just dumps everything into rows)
    if len(df) > 1:
        new_header = df.iloc[0]
        df = df[1:]
        df.columns = new_header
        
        # Set first column as index (e.g. "Índice de preços") to align rows across meetings
        # We assume the first column is the label
        if not df.empty:
            # Clean up index name
            df = df.set_index(df.columns[0])
            
            # Rename columns to include meeting number to avoid collisions
            # e.g. "2025" -> "2025 (M274)"
            df.columns = [f"{col} (M{meeting_num})" for col in df.columns]
            
            dfs_to_join.append(df)

if dfs_to_join:
    df_tables_horizontal = pd.concat(dfs_to_join, axis=1)
    print("All tables joined horizontally:")
    display(df_tables_horizontal)
else:
    print("No tables to join.")

All tables joined horizontally:


,2024 (M265),2025 (M265),1º tri 2026 (M265),2024 (M266),2025 (M266),2º tri 2026 (M266),2024 (M267),2025 (M267),2º tri 2026 (M267),2025 (M268),...,2026 (M271),2025 (M272),2026 (M272),1º tri 2027 (M272),2025 (M273),2026 (M273),1º tri 2027 (M273),2025 (M274),2026 (M274),2º tri 2027 (M274)
Índice de preços,,,,,,,,,,,,,,,,,,,,,
IPCA,"4,3","3,7","3,5","4,6","3,9","3,6","4,9","4,5","4,0","5,2",...,"3,6","4,9","3,6","3,4","4,8","3,6","3,4","4,6","3,6","3,3"
IPCA livres,"4,4","3,6","3,4","4,5","3,8","3,4","5,0","4,5","3,8","5,2",...,"3,4","5,1","3,5","3,3","5,0","3,5","3,3","4,5","3,6","3,2"
IPCA administrados,"4,2","4,0","3,9","4,9","4,2","4,3","4,6","4,5","4,6","5,2",...,"4,1","4,4","4,0","3,9","4,3","3,8","3,8","5,0","3,4","3,5"
